# Dataloader

> Support various dataloader for loading batches.

In [ ]:
#| default_exp data.loader

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
from nbdev import show_doc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.utils import get_config

In [ ]:
#| export
try:
    import torch.utils.data as torch_data
except ModuleNotFoundError:
    torch_data = None

## Dataset

In [ ]:
#| export
class Dataset(ABC):
    """A pytorch-like abstract Dataset class."""

    def __getitem__(self, index):
        raise NotImplementedError

In [ ]:
#| export
class ArrayDataset(Dataset):
    """Dataset wrapping tensors."""

    def __init__(
        self, 
        *arrays: jnp.DeviceArray # Numpy array with same first dimension
    ):
        assert all(arrays[0].shape[0] == arr.shape[0] for arr in arrays), \
            "All arrays must have the same dimension."
        self.arrays = arrays

    def __len__(self):
        return self.arrays[0].shape[0]

    def __getitem__(self, index):
        return tuple(arr[index] for arr in self.arrays)

In [ ]:
key = random.PRNGKey(0)
X = jax.random.normal(key, shape=(10, 10))
y = jax.random.normal(key, shape=(10, ))
ds = ArrayDataset(X, y)

assert len(ds) == 10

Indexing `Dataset` using `ds[idx]`

In [ ]:
x1, y1 = ds[1]
assert jnp.array_equal(x1, X[1])
assert jnp.array_equal(y1, y[1])

## Dataloader

In [ ]:
#| export
class BaseDataLoader(ABC):
    """Dataloader Interface"""
    def __init__(
        self, 
        dataset,
        backend: str,
        batch_size: int = 1,    # Batch size
        shuffle: bool = False,  # If true, dataloader shuffles before sampling each batch
        drop_last: bool = False, # Drop last batches or not
        **kwargs # Aux arguments
    ):
        pass 
    
    def __len__(self):
        raise NotImplementedError
    
    def __next__(self):
        raise NotImplementedError
    
    def __iter__(self):
        raise NotImplementedError

In [ ]:
#| hide
def test_dataloader(dataloader_cls, samples=1000, batch_size=12):
    feats = jnp.arange(samples).repeat(10).reshape(samples, 10)
    labels = jnp.arange(samples).reshape(samples, 1)
    ds = ArrayDataset(feats, labels)
    # N % batchsize != 0
    dl = dataloader_cls(ds, batch_size=batch_size, shuffle=False)
    for _ in range(2):
        X_list, Y_list = [], []
        for x, y in dl:
            X_list.append(x)
            Y_list.append(y)
        _X, _Y = map(jnp.concatenate, (X_list, Y_list))
        assert jnp.array_equal(_X, feats)
        assert jnp.array_equal(_Y, labels)

    dl = dataloader_cls(ds, batch_size=batch_size, shuffle=False, drop_last=True)
    for _ in range(2):
        X_list, Y_list = [], []
        for x, y in dl:
            X_list.append(x)
            Y_list.append(y)
        _X, _Y = map(jnp.concatenate, (X_list, Y_list))
        last_idx = len(X_list) * batch_size
        assert jnp.array_equal(_X, feats[: last_idx])
        assert jnp.array_equal(_Y, labels[: last_idx])


    dl_shuffle = dataloader_cls(ds, batch_size=batch_size, shuffle=True, drop_last=False)
    for _ in range(2):
        X_list, Y_list = [], []
        for x, y in dl_shuffle:
            assert jnp.array_equal(x[:, :1], y)
            X_list.append(x)
            Y_list.append(y)
        _X, _Y = map(jnp.concatenate, (X_list, Y_list))
        assert not jnp.array_equal(_X, feats)
        assert not jnp.array_equal(_Y, labels)
        assert jnp.sum(_X) == jnp.sum(feats), \
            f"jnp.sum(_X)={jnp.sum(_X)}, jnp.sum(feats)={jnp.sum(feats)}"


    dl_shuffle = dataloader_cls(ds, batch_size=batch_size, shuffle=True, drop_last=True)
    for _ in range(2):
        X_list, Y_list = [], []
        for x, y in dl_shuffle:
            assert jnp.array_equal(x[:, :1], y)
            X_list.append(x)
            Y_list.append(y)
        _X, _Y = map(jnp.concatenate, (X_list, Y_list))
        assert not jnp.array_equal(_X, feats)
        assert not jnp.array_equal(_Y, labels)
        assert len(_X) == len(X_list) * batch_size


In [ ]:
#| export
class JaxDataloader(BaseDataLoader):
    """Dataloder in vanilla Jax"""

    def __init__(
        self, 
        dataset: Dataset,
        backend: str = 'jax', # Position argument
        batch_size: int = 1,  # Batch size
        shuffle: bool = False,  # If true, dataloader shuffles before sampling each batch
        drop_last: bool = False, # Drop last batches or not
        **kwargs
    ):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last

        self.keys = hk.PRNGSequence(get_config().global_seed)
        self.data_len = len(dataset)  # Length of the dataset
        self.indices = jnp.arange(self.data_len) # available indices in the dataset
        self.pose = 0  # record the current position in the dataset
        self._shuffle()

    def _shuffle(self):
        if self.shuffle:
            self.indices = jax.random.permutation(next(self.keys), self.indices)
        
    def _stop_iteration(self):
        self.pose = 0
        self._shuffle()
        raise StopIteration

    def __len__(self):
        if self.drop_last:
            batches = len(self.dataset) // self.batch_size  # get the floor of division
        else:
            batches = -(len(self.dataset) // -self.batch_size)  # get the ceil of division
        return batches

    def __next__(self):
        if self.pose + self.batch_size <= self.data_len:
            batch_indices = self.indices[self.pose: self.pose + self.batch_size]
            batch_data = self.dataset[batch_indices, ...]
            self.pose += self.batch_size
            return batch_data
        elif self.pose < self.data_len and not self.drop_last:
            batch_indices = self.indices[self.pose:]
            batch_data = self.dataset[batch_indices, ...]
            self.pose += self.batch_size
            return batch_data
        else:
            self._stop_iteration()

    def __iter__(self):
        return self

In [ ]:
#| hide
test_dataloader(JaxDataloader, samples=100, batch_size=12)
test_dataloader(JaxDataloader, samples=100, batch_size=10)
test_dataloader(JaxDataloader, samples=101, batch_size=10)

In [ ]:
#| exporti
# copy from https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html#data-loading-with-pytorch
def _numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple, list)):
        transposed = zip(*batch)
        return [_numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)

def _convert_dataset_pytorch(dataset: Dataset):
    class _TorchDataset(torch_data.Dataset):
        def __init__(self, dataset: Dataset): self.dataset = dataset
        def __len__(self): return len(self.dataset)
        def __getitem__(self, idx): return self.dataset[idx]
    
    return _TorchDataset(dataset)

In [ ]:
#| export
class TorchDataloader(BaseDataLoader):
    """Use `Pytorch` to load batches. It requires [pytorch](https://pytorch.org/get-started/) to be installed."""
    
    def __init__(
        self, 
        dataset: Dataset,
        backend: str = 'pytorch', # positional argument
        batch_size: int = 1,  # batch size
        shuffle: bool = False,  # if true, dataloader shuffles before sampling each batch
        num_workers: int = 0, # number of workers
        drop_last: bool = False, # drop last batch or not
        **kwargs
    ):
        if torch_data is None:
            raise ModuleNotFoundError("`pytorch` library needs to be installed. Try `pip install torch`."
            "Please refer to pytorch documentation for details: https://pytorch.org/get-started/.")
        
        dataset = _convert_dataset_pytorch(dataset)
        self.dataloader = torch_data.DataLoader(
            dataset, 
            batch_size=batch_size, 
            shuffle=shuffle, 
            num_workers=num_workers, 
            drop_last=drop_last,
            collate_fn=_numpy_collate,
            **kwargs
        ) 

    def __len__(self):
        return len(self.dataloader)

    def __next__(self):
        return next(self.dataloader)

    def __iter__(self):
        return self.dataloader.__iter__()

In [ ]:
#| hide
test_dataloader(TorchDataloader, samples=100, batch_size=12)
# test_dataloader(TorchDataloader, samples=1000, batch_size=10)
# test_dataloader(TorchDataloader, samples=1001, batch_size=10)

## Main Dataloader Class

In [ ]:
#| export
@dataclass(frozen=True)
class DataloaderBackends:
    jax: BaseDataLoader = JaxDataloader
    pytorch: BaseDataLoader = TorchDataloader
    tensorflow: BaseDataLoader = None
    merlin: BaseDataLoader = None

    __all__ = dict(
        jax=jax, pytorch=pytorch, tensorflow=tensorflow, merlin=merlin
    )

    def __getitem__(self, key):
        return self.__all__[key]

    @classmethod
    def supported(cls) -> List[str]:
        return [
            backend for backend, dl_cls in cls.__all__.items() if dl_cls is not None
        ]

In [ ]:
#| exporti
def _dispatch_dataloader(
    backend: str # dataloader backend
) -> BaseDataLoader:
    """Return Dataloader class based on given `backend`"""

    backends = DataloaderBackends()
    if not backend in DataloaderBackends.supported():
        raise ValueError(f"backend=`{backend}` is either an invalid backend or not supported yet. "
            f"Should be one of {backends.supported}.")
    
    dl_cls = backends[backend]
    return dl_cls

In [ ]:
show_doc(_dispatch_dataloader)

---

[source](https://github.com/birkhoffg/relax/tree/master/blob/master/relax/data/loader.py#L176){target="_blank" style="float:right; font-size:smaller"}

### _dispatch_dataloader

>      _dispatch_dataloader (backend:str)

Return Dataloader class based on given `backend`

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| backend | str | dataloader backend |
| **Returns** | **BaseDataLoader** |  |

In [ ]:
assert _dispatch_dataloader('jax') == JaxDataloader
assert _dispatch_dataloader('pytorch') == TorchDataloader

In [ ]:
#| export
class DataLoader(BaseDataLoader):
    """Main Dataloader class to load Numpy data batches"""
    def __init__(
        self,
        dataset: Dataset,
        backend: str, # Dataloader backend; Currently supports `['jax', 'pytorch']`
        batch_size: int = 1,  # batch size
        shuffle: bool = False,  # if true, dataloader shuffles before sampling each batch
        num_workers: int = 0, # number of workers
        drop_last: bool = False, # drop last batches or not
        **kwargs
    ):
        self.__class__ = _dispatch_dataloader(backend)
        self.__init__(
            dataset=dataset, 
            backend=backend, 
            batch_size=batch_size, 
            shuffle=shuffle, 
            num_workers=num_workers,
            drop_last=drop_last,
            **kwargs
        )

#### A Minimum Example of using Dataloader

We showcase how to use `Dataloader` for training a simple regression model.


In [ ]:
from sklearn.datasets import make_regression

In [ ]:
X, y = make_regression(n_samples=10000, n_features=20)
dataset = ArrayDataset(X, y.reshape(-1, 1))
keys = hk.PRNGSequence(0)

Define `loss`, `step`, `train`:

In [ ]:
def loss(w, x, y):
    return jnp.mean(vmap(optax.l2_loss)(x @ w.T, y))

def step(w, x, y):
    lr = 0.1
    grad = jax.grad(loss)(w, x, y)
    w -= lr * grad
    return w

def train(dataloader: DataLoader, key: random.PRNGKey):
    w = jax.random.normal(key, shape=(1, 20))
    n_epochs = 10
    for _ in range(n_epochs):
        for x, y in dataloader:
            w = step(w, x, y)
    return w

def eval(dataloader: DataLoader, w):
    err = []
    for x, y in dataloader:
        err.append(loss(w, x, y))
    return np.mean(err)
    

Train this linear regression model via `DataLoaderJax`:

In [ ]:
dataloader = DataLoader(
    dataset, 'jax', batch_size=128, shuffle=True)
w = train(dataloader, next(keys)).block_until_ready()
assert np.allclose(eval(dataloader, w), 0.)


In [ ]:
#| include: false
dataloader = DataLoader(dataset, 'jax', batch_size=200, shuffle=True)
w = train(dataloader, next(keys))
assert np.allclose(eval(dataloader, w), 0.)

Train this linear regression model via `DataLoaderPytorch`:

In [ ]:
dataloader = DataLoader(
    dataset, 'pytorch', batch_size=128, shuffle=True)
w = train(dataloader, next(keys)).block_until_ready()
assert np.allclose(eval(dataloader, w), 0.)